In [1]:
import os
import pandas as pd
from IPython.display import clear_output

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)

In [3]:
RAW_FILEPATH = os.path.join('..', 'raw')

### Read in Voteview, DIME, and SCTV

#### Read in voteview** (https://voteview.com/articles/data_help_members)

In [4]:
VV_IDENTIFIERS = ['icpsr', 'bioname', 'congress', 'chamber']
VV_IDEO = ['nominate_dim1', 'nominate_log_likelihood', 'nominate_geo_mean_probability', 'nokken_poole_dim1']
VV_USECOLS = VV_IDENTIFIERS + VV_IDEO

vv = pd.read_csv(os.path.join(RAW_FILEPATH, 'HSall_members.csv'), usecols=VV_USECOLS)

vv['icpsr'] = vv['icpsr'].astype('str')

vv.sort_values(by=['congress', 'icpsr'], ascending=True, inplace=True)

# Fill forward, retain latest scores where available
for col in  VV_IDEO:
    vv[col] = vv.groupby('bioname')[col].apply(lambda x: x.ffill())

# Retain lastest obs
vv = vv.groupby('icpsr').tail(1)

#### Read in DIME

In [5]:
DIME_IDENTIFIERS = ['cycle', 'name', 'ICPSR2', 'party', 'state', 'seat', 'district']
DIME_IDEOLOGIES = ['recipient.cfscore', 'contributor.cfscore', 'dwnom1']
DIME_USECOLS = DIME_IDENTIFIERS + DIME_IDEOLOGIES

dime = pd.read_csv(os.path.join(RAW_FILEPATH, 'dime_recipients_1979_2014.csv'), usecols=DIME_USECOLS)

# # Drop committees (cf codebook p. 17)
# dime = dime[~dime['seat'].isin(['federal:committee', 'state:committee', 'federal:527'])]

dime.columns = [col.lower() for col in dime.columns]

dime.sort_values(by=['cycle', 'icpsr2'], ascending=True, inplace=True)

# Fill forward, retain latest scores where available
for col in  DIME_IDEOLOGIES:
    dime[col] = dime.groupby('name')[col].apply(lambda x: x.ffill())

# Remove nominee from ICPSR (only for seem fpr presidential candidates)
dime['icpsr2'] = [s.replace('nominee', '') for s in dime['icpsr2']]

# Retain last
dime = dime.groupby('icpsr2').tail(1)

dime.rename(columns={'icpsr2': 'icpsr'}, inplace=True)

# Ideology coding
ideo_dict = {100: 'D', 200: 'R', 328: 'I'}
dime['party'] = dime['party'].map(ideo_dict)

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (18,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Updated 2018 DIME for congress
DIME_UPDATED_IDENTIFIERS = ['cycle', 'Name', 'ICPSR2', 'party', 'state', 'seat', 'district']
DIME_UPDATED_IDEOLOGIES = ['recipient_cfscore', 'contributor_cfscore', 'dwnom1']
DIME_UPDATED_USECOLS = DIME_UPDATED_IDEOLOGIES + DIME_UPDATED_IDENTIFIERS

dime_updated = pd.read_csv(os.path.join(RAW_FILEPATH, 'dime_cong_elections_current.csv'), usecols=DIME_UPDATED_USECOLS)

# Drop those before 2014
dime_updated = dime_updated[dime_updated['cycle']>2014]

# # Drop committees (cf codebook p. 17)
# dime_updated = dime_updated[~dime_updated['seat'].isin(['federal:committee', 'state:committee', 'federal:527'])]

dime_updated.columns = [col.lower() for col in dime_updated.columns]

dime_updated.sort_values(by=['cycle', 'icpsr2'], ascending=True, inplace=True)

# Fill forward, retain latest scores where available
for col in  DIME_UPDATED_IDEOLOGIES:
    dime_updated[col] = dime_updated.groupby('name')[col].apply(lambda x: x.ffill())

# Remove nominee from ICPSR (only for seem fpr presidential candidates)
dime_updated['icpsr2'] = [s.replace('nominee', '') for s in dime_updated['icpsr2']]

# Retain last
dime_updated = dime_updated.groupby('icpsr2').tail(1)

dime_updated.rename(columns={'icpsr2': 'icpsr',
                             'recipient_cfscore': 'recipient.cfscore', 
                             'contributor_cfscore': 'contributor.cfscore'}, inplace=True)

In [7]:
dime = dime.append(dime_updated, ignore_index=True, sort=True)

dime = dime.groupby('icpsr').tail(1)

dime['name'] = dime['name'].str.lower()
len(dime)

88123

#### Read in SCTV

In [8]:
# # find icpsr using name
# sctv = pd.read_csv(os.path.join(RAW_FILEPATH, 'sctvna.csv'), encoding = "ISO-8859-1")

# sctv.columns = ['name', 'presenter', 'minutes']

# # drop presenters
# sctv = sctv[sctv['presenter'].isnull()]
# sctv.drop('presenter', axis=1, inplace=True)

# sctv.reset_index(inplace=True, drop=True)
#-------------------------------------------------------------------------------------------------
# name_to_icpsr = dict()

# sctv['status'] = ''

# _df = df[['vv_name', 'seat', 'icpsr']]
# _df2 = df[['dime_name', 'seat', 'icpsr']]

# for ix in range(len(sctv)):
#     if pd.notna(sctv.loc[ix, 'icpsr']):
#         continue
        
#     if ix<1073:
#         continue
                
#     name = sctv.loc[ix, 'name']
#     clear_output(wait=True)
#     print(f"{ix}/1225: politician is {name}")
#     print(_df[_df['vv_name'].str.lower().str.contains(name.split(' ')[-1], na=False)])
#     print('--------------------------')
#     print(_df2[_df2['dime_name'].str.lower().str.contains(name.split(' ')[-1], na=False)])
    
# #     icpsr = input("ICPSR no. = ")
    
#     if icpsr=='br':
#         break
#     name_to_icpsr.update({name: icpsr})
    
#     if icpsr=='?':
#         sctv.loc[ix, 'status'] = 'no'
#     else:
#         sctv.loc[ix, 'status'] = 'yes'


In [9]:
sctv = pd.read_csv('name-to-icpsr.csv')
sctv.columns = ['sctv_name', 'minutes', 'icpsr']
len(sctv[sctv['icpsr'].notnull()])

388

### Merge

In [10]:
df = dime.merge(vv, how='outer', on='icpsr')

df.rename(columns={'name': 'dime_name', 'bioname': 'vv_name'}, inplace=True)

In [11]:
_pre_merge_count = len(sctv)

df = df.merge(sctv, how='right', on='icpsr')

assert _pre_merge_count==len(df)

In [13]:
df.to_csv(os.path.join('files', 'for-plots.csv'), index=False)